# Pre-processing CO Allocation data for WaDE upload.
- Purpose:  To pre-process the data into one master file for simple DataFrame creation and extraction

In [1]:
# Needed Libraries / Modules

# ---- working with data ----
import os  # native operating system interaction
import numpy as np  # mathematical array manipulation
import pandas as pd  # data structure and data analysis
import geopandas as gpd  # geo-data structure and data analysis

# ---- visualization ----
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library

# ---- API data retrieval ----
import requests  # http requests
import json  # JSON parse

# ---- Cleanup ----
import re  # string regular expression manipulation
from datetime import datetime  # date and time manipulation
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x)  # suppress scientific notation in Pandas

In [2]:
# ---- working directory ----
workingDirString = "G:/Shared drives/WaDE Data/Colorado/WaterAllocation" # set working directory folder string here

os.chdir(workingDirString)
print(f'The working Directory is:', workingDirString)

The working Directory is: G:\Shared drives\WaDE Data\Colorado\WaterAllocation


In [3]:
# Input File
fileInput = "RawInputData/DWR_Water_Right_-_Net_Amounts.zip"
dfinPOD = pd.read_csv(fileInput).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "coD" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('RawInputData/DWR_Water_Right_-_Net_Amounts.zip', compression=dict(method='zip', archive_name='DWR_Water_Right_-_Net_Amounts.csv'), index=False)

print(len(dfinPOD))
dfinPOD.head(1)

C:\Users\rjame\AppData\Local\Temp\ipykernel_15868\2522720005.py:3: DtypeWarning: Columns (12,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOD = pd.read_csv(fileInput).replace(np.nan, "")


171214


,WDID,Structure Name,Structure Type,Water Source,GNIS ID,Stream Mile,DIV,WD,County,Q10,Q40,Q160,Section,Township,Range,PM,CoordsEW,CoordsEW Dir,CoordsNS,CoordsNS Dir,UTM x,UTM y,Latitude,Longitude,Location Accuracy,Adjudication Date,Previous Adj Date,Appropriation Date,Admin No,Order No,Priority No,Associated Case Numbers,Decreed Uses,Net Absolute,Net Conditional,Net APEX Absolute,Net APEX Conditional,Decreed Units,Seasonal Limits,Comments,Modified,More Information,Location,WaDEUUID
0,103860,MILLAGE RES,Reservoir,SOUTH PLATTE RIVER,201759.00000,167.82000,1,1,WELD,,NW,SW,3.00000,5.0 N,64.0 W,S,,,,,538747.00000,4475216.00000,40.42668,-104.54323,GPS,12/31/1977,12/31/1976,03/31/1904,46386.19813,0,,W8623,1,150.00000,0.00000,0.00000,0.00000,A,No,,02/28/1992 12:00:00 AM,https://dwr.state.co.us/Tools/WaterRights/NetA...,"(40.426683, -104.543229)",coD0


In [4]:
#Creating Beneficial Use.
#Need to split CO abbreviatoin strings to a workable format.

BenUseDict = {
"0" : "Storage",
"1" : "Irrigation",
"2" : "Municipal",
"3" : "Commercial",
"4" : "Industrial",
"5" : "Recreation",
"6" : "Fishery",
"7" : "Fire",
"8" : "Domestic",
"9" : "Stock",
"A" : "Augmentation",
"B" : "Export from Basin",
"C" : "Cumulative Accretion to River",
"D" : "Cumulative Depletion from River",
"E" : "Evaporative",
"F" : "Federal Reserved",
"G" : "Geothermal",
"H" : "Household Use Only",
"K" : "Snow Making",
"M" : "Minimum Streamflow",
"N" : "Net Effect on River",
"P" : "Power Generation",
"Q" : "Other",
"R" : "Recharge",
"S" : "Export from State",
"T" : "Transmountain Export",
"W" : "Wildlife",
"X" : "All Beneficial Uses"}

def retrieveBenUse(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outString = ""
    else:
        String1 = str(colrowValue).strip()
        x=[]
        x[:] = String1
        try:
            outList = []
            for i in range(len(x)):
                y = x[i].strip()
                y = BenUseDict[y]
                outList.append(y)
            outString = ",".join(str(e) for e in outList)
        except:
            outString = ""
    return outString

dfinPOD['in_BeneficialUseCategory'] = dfinPOD.apply(lambda row: retrieveBenUse(row['Decreed Uses']), axis=1)
dfinPOD['in_BeneficialUseCategory'].unique()

array(['Irrigation', 'Irrigation,Stock',
       'Irrigation,Recreation,Fishery,Augmentation,Recharge,Wildlife',
       ..., 'Storage,Recreation,Fire,Wildlife',
       'Fire,Augmentation,Evaporative,Wildlife',
       'Storage,Recreation,Fire,Augmentation,Wildlife'], dtype=object)

In [5]:
#Determining WaterSourceTypeCV

WSTypeDict = {
"Aquifer NNT/NT Reservation" : "Surface Water",
"Ditch" : "Surface Water",
"Ditch System" : "Surface Water",
"Exchange Plan" : "Surface Water",
"Measuring Point" : "Surface Water",
"Mine" : "Surface Water",
"Minimum Flow" : "Surface Water",
"Other" : "Surface Water",
"Pipeline" : "Surface Water",
"Power Plant" : "Surface Water",
"Pump" : "Surface Water",
"Reach" : "Surface Water",
"Reach (Aggregating)" : "Surface Water",
"Recharge Area" : "Surface Water",
"Recharge Area Group" : "Surface Water",
"Reservoir" : "Surface Water",
"Reservoir System" : "Surface Water",
"Seep" : "Surface Water",
"Spring" : "Surface Water",
"Stream Gage" : "Surface Water",
"Well" : "Groundwater",
"Well Field" : "Groundwater",
"Well Group" : "Groundwater",
"Augmentation/Replacement Plan" : "Groundwater"}

def retrieveWaterSourceTypeCV(colrowValue):
    if (colrowValue == "") or (pd.isnull(colrowValue)):
        outString = "WaDE Blank"
    else:
        colrowValue = str(colrowValue).strip()
        try:
            outString = WSTypeDict[colrowValue]
        except:
            outString = "WaDE Blank"
    return outString

dfinPOD['in_WaterSourceTypeCV'] = dfinPOD.apply(lambda row: retrieveWaterSourceTypeCV(row['Structure Type']), axis=1)
dfinPOD['in_WaterSourceTypeCV'].unique()

array(['Surface Water', 'Groundwater', 'WaDE Blank'], dtype=object)

In [6]:
# #Determining WaterSourceName
# #Want to remove the "GROUNDWATER: " from the name.

# def determineWaterSourceName(colrowValue):
#     if colrowValue == "" or pd.isnull(colrowValue):
#         outString = ""
#     else:
#         String1 = str(colrowValue).strip()
#         if "GROUNDWATER:" in String1:
#             outString = colrowValue.replace("GROUNDWATER: ", "").strip()
#         else:
#             outString = colrowValue
#     return outString

# dfinPOD['in_WaterSourceName'] = dfinPOD.apply(lambda row: determineWaterSourceName(row['Water Source']), axis=1)
# dfinPOD['in_WaterSourceName'].unique()

In [7]:
# Allocation_CFS
# If Decreed Units = "C" and Net Absolute != 0, then return Net Absolute
# Elif Decreed Units = "C" and Net Conditional != 0, then return Net Conditional
# Else return blank

# For creating Allocation_CFS
def assignAllocation_CFS(valA, valB, valC):
    valA = str(valA).strip()
    if (valB != 0) and (valC != 0):
        outString = 0
    else:
        if (valA == "C") and (valB != 0):
            outString = valB
        elif (valA  == "C") and (valC != 0):
            outString = valC
        else:
            outString = 0
    return outString

dfinPOD['in_AllocationFlow_CFS'] = dfinPOD.apply(lambda row: assignAllocation_CFS(row["Decreed Units"], row["Net Absolute"], row["Net Conditional"]), axis=1)
dfinPOD['in_AllocationFlow_CFS'].unique()

array([0.000e+00, 5.330e-01, 4.450e-01, ..., 1.114e+03, 7.810e-01,
       2.825e+01])

In [8]:
# AllocationVolume_AF
# If Decreed Units = "A" and Net Absolute != 0, then return Net Absolute
# Elif Decreed Units = "A" and Net Conditional != 0, then return Net Conditional
# Else return blank

# For creating AllocationVolume_AF
def assignAllocationVolume_AF(valA, valB, valC):
    valA = str(valA).strip()
    if (valB != 0) and (valC != 0):
        outString = 0
    else:
        if (valA == "A") and (valB != 0):
            outString = valB
        elif (valA  == "A") and (valC != 0):
            outString = valC
        else:
            outString = 0
    return outString

dfinPOD['in_AllocationVolume_AF'] = dfinPOD.apply(lambda row: assignAllocationVolume_AF(row["Decreed Units"], row["Net Absolute"], row["Net Conditional"]), axis=1)
dfinPOD['in_AllocationVolume_AF'].unique()

array([1.50e+02, 8.25e+01, 0.00e+00, ..., 8.77e+00, 2.03e+04, 7.91e+02])

In [9]:
# For creating AllocationLegalStatusCV
# If Net Absolute = 0 and Net Condontial = 0, then Condtional Aboslute
# Elif Net Absolute = 0 and Net Condontial != 0, then Condtional
# Else, Aboslute

def assignAllocationLegalStatusCV(valA, valB):
    if (valA == 0) and (valB == 0):
        outString = "Conditional Absolute"
    elif (valA == 0) and (valB != 0):
        outString = "Conditional"
    else:
        outString = "Absolute"
    return outString

dfinPOD['in_AllocationLegalStatusCV'] = dfinPOD.apply(lambda row: assignAllocationLegalStatusCV(row['Net Absolute'], row['Net Conditional']), axis=1)
dfinPOD['in_AllocationLegalStatusCV'].unique()

array(['Absolute', 'Conditional Absolute', 'Conditional'], dtype=object)

In [10]:
# Need a unique identifier for WaDE AllocationNativeID.  Combine combine **Admin No**, **Order No**, **Decreed Units**, & **WDID** into single string entry.

# For creating AllocationAmount
def assignAllocationNativeID(colrowValueA, colrowValueB, colrowValueC, colrowValueD):
    colrowValueA = str(colrowValueA).strip()
    colrowValueB = str(colrowValueB).strip()
    colrowValueD = str(colrowValueD).strip()
    outString = "-".join(map(str, [colrowValueA, colrowValueB, colrowValueC, colrowValueD]))
    return outString

dfinPOD['in_AllocationNativeID'] = dfinPOD.apply(lambda row: assignAllocationNativeID(row['Admin No'], row['Order No'], row['Decreed Units'], row['WDID']), axis=1)
dfinPOD['in_AllocationNativeID'].unique()

array(['46386.19813-0-A-103860', '46020.20608-0-A-103894',
       '39446.0-0-C-105485', ..., '62912.0-0-C-7100733',
       '62912.0-0-A-7105073', '62912.0-0-A-7105071'], dtype=object)

In [11]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOD['WaDEUUID']

# Method Info
df['in_MethodUUID'] = "COwr_M1"

# Variable Info
df['in_VariableSpecificUUID'] = "COwr_V1"

# Organization Info
df['in_OrganizationUUID'] = "COwr_O1"

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOD['Water Source'].str.title()
df['in_WaterSourceNativeID'] = "" # auto fill in below
df['in_WaterSourceTypeCV'] = dfinPOD['in_WaterSourceTypeCV']

# Site Info
df['in_RegulatoryOverlayUUIDs'] = ""
df['in_WaterSourceUUID'] = "" # ???
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = dfinPOD['Location Accuracy']
df['in_County'] = dfinPOD['County'].str.title()
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfinPOD['Latitude']
df['in_Longitude'] = dfinPOD['Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfinPOD['Structure Name'].str.title()
df['in_SiteNativeID'] = dfinPOD['WDID'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = dfinPOD['Structure Type'].astype(str)
df['in_StateCV'] = "CO"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfinPOD['in_AllocationFlow_CFS'].astype(float) # see above for conversion
df['in_AllocationLegalStatusCV'] = dfinPOD['in_AllocationLegalStatusCV']
df['in_AllocationNativeID'] =  dfinPOD['in_AllocationNativeID'].astype(str)
df['in_AllocationOwner'] = "WaDE Unspecified"
df['in_AllocationPriorityDate'] = dfinPOD['Appropriation Date']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = "12/31"
df['in_AllocationTimeframeStart'] = "01/01"
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfinPOD['in_AllocationVolume_AF'].astype(float) # see above for conversion
df['in_BeneficialUseCategory'] = dfinPOD['in_BeneficialUseCategory']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = dfinPOD['More Information']

df = df.drop_duplicates().reset_index(drop=True)
print(len(df))
df.head()

171214


,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_RegulatoryOverlayUUIDs,in_WaterSourceUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,coD0,COwr_M1,COwr_V1,COwr_O1,,,,South Platte River,,Surface Water,,,,GPS,Weld,4326,,,,40.42668,-104.54323,,,POD,Millage Res,103860,,Reservoir,CO,,,,,,,,,,0.00000,Absolute,46386.19813-0-A-103860,WaDE Unspecified,03/31/1904,,12/31,01/01,,150.00000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
1,coD1,COwr_M1,COwr_V1,COwr_O1,,,,Willow Creek,,Surface Water,,,,Digitized,Weld,4326,,,,40.53346,-104.56242,,,POD,Mason Res,103894,,Reservoir,CO,,,,,,,,,,0.00000,Absolute,46020.20608-0-A-103894,WaDE Unspecified,06/04/1906,,12/31,01/01,,82.50000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
2,coD2,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: South Platte River,,Groundwater,,,,GPS,Weld,4326,,,,40.61786,-104.66634,,,POD,Andersen Well 2-12608R,105485,,Well,CO,,,,,,,,,,0.53300,Absolute,39446.0-0-C-105485,WaDE Unspecified,12/31/1957,,12/31,01/01,,0.00000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
3,coD3,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: South Platte River,,Groundwater,,,,GPS,Weld,4326,,,,40.62515,-104.67519,,,POD,Andersen Well 7057,105489,,Well,CO,,,,,,,,,,0.44500,Absolute,31529.0-0-C-105489,WaDE Unspecified,04/28/1936,,12/31,01/01,,0.00000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
4,coD4,COwr_M1,COwr_V1,COwr_O1,,,,Lost Creek,,Surface Water,,,,Digitized,Weld,4326,,,,40.12613,-104.37217,,,POD,Shoeneman Holding Pond 2,103884,,Reservoir,CO,,,,,,,,,,0.00000,Absolute,55882.3068-0-A-103884,WaDE Unspecified,12/31/1933,,12/31,01/01,,6.00000,"Irrigation,Stock",,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...


In [12]:
# Concatenate dataframes
frames = [df] # add dataframes here
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

171214


## Clean output dataframes

In [13]:
# Clean owner name up
def cleanDataFunc(Val):
    Val = re.sub("[$@&.;,/\)(-]", "", Val).strip()
    return Val

In [14]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: cleanDataFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

array(['WaDE Unspecified'], dtype=object)

In [15]:
outdf['in_SiteName'] = outdf.apply(lambda row: cleanDataFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['Millage Res', 'Mason Res', 'Andersen Well 212608R', ...,
       'Homestake Well No 3', 'Garcia Well', 'Homestake Pond Diversion'],
      dtype=object)

In [16]:
# Ensure Empty String

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [17]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['South Platte River', 'Willow Creek',
       'Groundwater: South Platte River', ...,
       'South Branch Boxelder Creek', 'Groundwater: East Beaver Creek',
       'Groundwater: Fourteenmile Creek'], dtype=object)

In [18]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['Surface Water', 'Groundwater', 'WaDE Blank'], dtype=object)

In [19]:
outdf['in_County'] = outdf.apply(lambda row: ensureEmptyString(row['in_County']), axis=1)
outdf['in_County'].unique()

array(['Weld', 'Morgan', 'Adams', 'Elbert', 'Arapahoe', 'El Paso',
       'Washington', 'Douglas', 'Denver', 'Jefferson', 'Broomfield',
       'Lincoln', 'Larimer', 'Unknown', 'Boulder', 'Grand', 'Gilpin',
       'Clear Creek', 'Teller', 'Conejos', 'Park', 'Lake', 'Chaffee',
       'Huerfano', 'Otero', 'Las Animas', '', 'Pueblo', 'Fremont',
       'Custer', 'Saguache', 'Pitkin', 'Crowley', 'Bent', 'Kiowa',
       'Costilla', 'Mineral', 'Rio Grande', 'Alamosa', 'Hinsdale',
       'San Juan', 'Archuleta', 'Phillips', 'Gunnison', 'La Plata',
       'Montezuma', 'Dolores', 'San Miguel', 'Summit', 'Eagle',
       'Garfield', 'Delta', 'Rio Blanco', 'Mesa', 'Routt', 'Montrose',
       'Moffat', 'Ouray', 'Jackson', 'Kit Carson', 'Yuma', 'Cheyenne',
       'Logan', 'Sedgwick', 'Prowers', 'Baca'], dtype=object)

In [20]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['Millage Res', 'Mason Res', 'Andersen Well 212608R', ...,
       'Homestake Well No 3', 'Garcia Well', 'Homestake Pond Diversion'],
      dtype=object)

In [21]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

array(['Reservoir', 'Well', 'Ditch', 'Spring', 'Recharge Area', 'Other',
       'Pipeline', 'Well Field', 'Pump', 'Seep', 'Reservoir System',
       'Reach', 'Well Group', 'Minimum Flow', 'Mine', 'Power Plant',
       'Measuring Point', 'Augmentation/Replacement Plan',
       'Recharge Area Group', 'Stream Gage', 'Exchange Plan',
       'Ditch System', 'Reach (Aggregating)',
       'Aquifer NNT/NT Reservation', 'Entity', 'Stream Confluence',
       'Livestock Water Tank'], dtype=object)

In [22]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['Absolute', 'Conditional Absolute', 'Conditional'], dtype=object)

In [23]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'])
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

<DatetimeArray>
['1904-03-31 00:00:00', '1906-06-04 00:00:00', '1957-12-31 00:00:00',
 '1936-04-28 00:00:00', '1933-12-31 00:00:00', '1888-10-01 00:00:00',
 '1868-04-20 00:00:00', '1934-06-20 00:00:00', '1933-09-20 00:00:00',
 '1955-04-14 00:00:00',
 ...
 '2004-06-22 00:00:00', '2022-12-22 00:00:00', '1872-06-30 00:00:00',
 '2021-05-13 00:00:00', '2021-11-22 00:00:00', '2016-06-18 00:00:00',
 '2023-05-10 00:00:00', '2023-01-24 00:00:00', '1995-07-14 00:00:00',
 '2022-10-28 00:00:00']
Length: 25486, dtype: datetime64[ns]

In [24]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

array(['', 0.533, 0.445, ..., 1114.0, 0.781, 28.25], dtype=object)

In [25]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

array([150.0, 82.5, '', ..., 8.77, 20300.0, 791.0], dtype=object)

In [26]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna("")
outdf['in_Latitude'].unique()

array([40.426683, 40.53346, 40.617859, ..., 37.703248, 39.304927,
       37.707755], dtype=object)

In [27]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna("")
outdf['in_Longitude'].unique()

array([-104.543229, -104.562424, -104.666336, ..., -108.031428,
       -106.260375, -108.032757], dtype=object)

In [28]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['wadeID1', 'wadeID2', 'wadeID3', ..., 'wadeID3247', 'wadeID3248',
       'wadeID3249'], dtype=object)

## Drop non-Active AllocationLegalStatusCV Water Rights
- For this CO, we don't want water rights that are considered: Conditional

In [29]:
# drop non-active AllocationLegalStatusCV values specific to that state.

# drop the list
dropLegalStatusList = ["Conditional"] # enter string entries here

# drop rows from above list
outdf = outdf[outdf.in_AllocationLegalStatusCV.isin(dropLegalStatusList) == False].reset_index(drop=True)

print(len(outdf))
outdf['in_AllocationLegalStatusCV'].unique()

161315


array(['Absolute', 'Conditional Absolute'], dtype=object)

## Export Outputs

In [30]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(outdf.dtypes)

WaDEUUID                                                object
in_MethodUUID                                           object
in_VariableSpecificUUID                                 object
in_OrganizationUUID                                     object
in_Geometry                                             object
in_GNISFeatureNameCV                                    object
in_WaterQualityIndicatorCV                              object
in_WaterSourceName                                      object
in_WaterSourceNativeID                                  object
in_WaterSourceTypeCV                                    object
in_RegulatoryOverlayUUIDs                               object
in_WaterSourceUUID                                      object
in_CoordinateAccuracy                                   object
in_CoordinateMethodCV                                   object
in_County                                               object
in_EPSGCodeCV                                          

In [31]:
outdf

,WaDEUUID,in_MethodUUID,in_VariableSpecificUUID,in_OrganizationUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_RegulatoryOverlayUUIDs,in_WaterSourceUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDate,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationSDWISIdentifierCV,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,coD0,COwr_M1,COwr_V1,COwr_O1,,,,South Platte River,wadeID1,Surface Water,,,,GPS,Weld,4326,,,,40.42668,-104.54323,,,POD,Millage Res,103860,,Reservoir,CO,,,,,,,,,,,Absolute,46386.19813-0-A-103860,WaDE Unspecified,1904-03-31,,12/31,01/01,,150.00000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
1,coD1,COwr_M1,COwr_V1,COwr_O1,,,,Willow Creek,wadeID2,Surface Water,,,,Digitized,Weld,4326,,,,40.53346,-104.56242,,,POD,Mason Res,103894,,Reservoir,CO,,,,,,,,,,,Absolute,46020.20608-0-A-103894,WaDE Unspecified,1906-06-04,,12/31,01/01,,82.50000,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
2,coD2,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: South Platte River,wadeID3,Groundwater,,,,GPS,Weld,4326,,,,40.61786,-104.66634,,,POD,Andersen Well 212608R,105485,,Well,CO,,,,,,,,,,0.53300,Absolute,39446.0-0-C-105485,WaDE Unspecified,1957-12-31,,12/31,01/01,,,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
3,coD3,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: South Platte River,wadeID3,Groundwater,,,,GPS,Weld,4326,,,,40.62515,-104.67519,,,POD,Andersen Well 7057,105489,,Well,CO,,,,,,,,,,0.44500,Absolute,31529.0-0-C-105489,WaDE Unspecified,1936-04-28,,12/31,01/01,,,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
4,coD4,COwr_M1,COwr_V1,COwr_O1,,,,Lost Creek,wadeID4,Surface Water,,,,Digitized,Weld,4326,,,,40.12613,-104.37217,,,POD,Shoeneman Holding Pond 2,103884,,Reservoir,CO,,,,,,,,,,,Absolute,55882.3068-0-A-103884,WaDE Unspecified,1933-12-31,,12/31,01/01,,6.00000,"Irrigation,Stock",,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161310,coD171201,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: South Platte River,wadeID3,Groundwater,,,,GPS,Logan,4326,,,,40.57040,-103.37314,,,POD,Brunkhardt Well 66513F,6405224,,Well,CO,,,,,,,,,,2.15000,Absolute,42154.0-0-C-6405224,WaDE Unspecified,1965-05-31,,12/31,01/01,,,Irrigation,,,,,,0,,,,,,,,,,https://dwr.state.co.us/Tools/WaterRights/NetA...
161311,coD171202,COwr_M1,COwr_V1,COwr_O1,,,,Groundwater: Arkansas River,wadeID395,Groundwater,,,,GPS,Pueblo,4326,,,,38.25957,-104.39391,,,POD,Solada Well No 2,1405143,,Well,CO,,,,,,,,,,3.02000,Absolute,31410.0-0-C-1405143,WaDE Unspecified,1935-12-31,,12/31,01/01,,,"Irrigation,Industrial",,,,,,0,,,,,,,,,,https://dwr.state.co.us

In [32]:
# Export the output dataframe
outdf.to_csv('RawInputData/Pwr_coMain.zip', compression=dict(method='zip', archive_name='Pwr_coMain.csv'), index=False)  # The output, save as a zip